In [135]:
import requests
from requests_html import HTML
import time
import pandas as pd

In [82]:
base_url = 'https://stackoverflow.com/questions/tagged/'
tag = 'python'
query_filter = 'Votes'
url = f'{base_url}{tag}?tab={query_filter}'
url

'https://stackoverflow.com/questions/tagged/python?tab=Votes'

In [83]:
r = requests.get(url)
html_text = r.text
html = HTML(html=html_text)

In [84]:
question_summary = html.find('.question-summary')

In [85]:
# print(question_summary)
# print(question_summary[2].text)

In [86]:
# columns = ['votes', 'vote_title', 'num_answers', 'views', 'question', 'short_desc', 'tags', 'date', 'user', 'user_detail']
# this_row = list(question_summary[0].text.split('\n'))
# row_data = dict(zip(columns, this_row))

In [87]:
key_names = ['question', 'votes', 'tags ']
classes_needed = ['.question-hyperlink', '.vote', '.tags']
this_question_element = question_summary[0]
this_question_element.find('.question-hyperlink', first=True).text
this_question_element.find('.vote', first=True).text.replace('\nvotes', '')

'10464'

In [88]:
def clean_scraped_data(text, keyname=None):
    if keyname == 'votes':
        return text.replace('\nvotes', '')
    return text

In [89]:
datas = []

for q_el in question_summary:
    question_data = {}
    for i, _class in enumerate(classes_needed):
        sub_el = q_el.find(_class, first=True)
        keyname = key_names[i]
        question_data[keyname] = clean_scraped_data(sub_el.text, keyname=keyname)
    datas.append(question_data)

In [116]:
def parse_tagged_page(html):
    question_summary = html.find('.question-summary')
    key_names = ['question', 'votes', 'tags ']
    classes_needed = ['.question-hyperlink', '.vote', '.tags']
    datas = []
    for q_el in question_summary:
        question_data = {}
        for i, _class in enumerate(classes_needed):
            sub_el = q_el.find(_class, first=True)
            keyname = key_names[i]
            question_data[keyname] = clean_scraped_data(sub_el.text, keyname=keyname)
        datas.append(question_data)
    return datas

In [142]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return []
    html_text = r.text
    html = HTML(html=html_text)
    dat = parse_tagged_page(html)
    return dat

In [143]:
def scrape_tag(tag = 'python', query_filter = 'Votes', pagesize=10, max_page=1):
    base_url = 'https://stackoverflow.com/questions/tagged/'
    datas = []
    for p in range(max_page):
        page_num = p + 1
        url = f'{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}'
        datas += extract_data_from_url(url)
        time.sleep(1.3)
    return datas

In [144]:
datas = scrape_tag(max_page=1)

In [145]:
len(datas)

50

In [146]:
df = pd.DataFrame(datas)
df.head()

,question,votes,tags
0,What does the “yield” keyword do?,10464,python iterator generator yield coroutine
1,What does if __name__ == “__main__”: do?,6262,python namespaces main python-module idioms
2,Does Python have a ternary conditional operator?,6202,python operators ternary-operator conditional-...
3,What are metaclasses in Python?,5833,python oop metaclass python-datamodel
4,How do I check whether a file exists without e...,5740,python file file-exists


In [148]:
df.to_csv("python.csv", index=False)